In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

In [4]:
features = [ 'Age','NCP']



for feature in features:
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Filter out outliers
    df = df[(df[feature] >= lower) & (df[feature] <= upper)]

In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
df.drop_duplicates(inplace=True)

In [7]:
# from sklearn.preprocessing import LabelEncoder

# # list of categorical columns
# categorical_columns = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC',
#                        'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad']

# # apply label encoding to each column
# le = LabelEncoder()
# for col in categorical_columns:
#     df[col] = le.fit_transform(df[col])

labelencoder = LabelEncoder()
object_columns = df.select_dtypes(include='object').columns.difference(['NObeyesdad'])

encoders = {}
for col_name in object_columns:
    le = LabelEncoder()
    df[col_name] = le.fit_transform(df[col_name])
    encoders[col_name] = le  # store individual encoder

# Encode target
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(df['NObeyesdad'])
import joblib
# Save encoders
joblib.dump(encoders, 'encoders.pkl')
joblib.dump(target_encoder, 'target_encoder.pkl')


['target_encoder.pkl']

['Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III', 'Overweight_Level_I', 'Overweight_Level_II']


In [8]:
!pip install concrete_ml

In [11]:
X = df.drop(['NObeyesdad'], axis=1)
y = df['NObeyesdad']
y = labelencoder.fit_transform(y)

print(X.shape)
print(y.shape)
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape , y_train.shape, X_val.shape, y_val.shape


(1401, 16)
(1401,)


((1120, 16), (1120,), (281, 16), (281,))

In [13]:


optimal_param = {'max_depth': 3, 'n_bits': 6, 'n_estimators': 5}


concrete_xgb_clf = ConcreteXGBClassifier(**optimal_param)

concrete_xgb_clf.fit(X_train, y_train)

fhe_circuit = concrete_xgb_clf.compile(X_train)

fhe_circuit.client.keygen(force=False)

In [16]:
from pathlib import Path
from concrete.ml.deployment import FHEModelDev

fhe_directory = Path("./health_prediction/deployment").resolve()

dev = FHEModelDev(path_dir=fhe_directory, model=concrete_xgb_clf)
dev.save()

In [9]:
from sklearn.model_selection import GridSearchCV
from concrete.ml.sklearn import XGBClassifier as ConcreteXGBClassifier


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [12]:
param_grid = {
    "n_bits": [2, 3, 4, 5, 6, 7],
    "max_depth": [1, 2, 3],
    "n_estimators": [1, 2, 3, 5],
}
concrete_clf = ConcreteXGBClassifier(n_jobs=-1)
grid_search = GridSearchCV(
    concrete_clf,
    param_grid,
    cv=4,
    verbose=1,
    n_jobs=-1,
).fit(X_train, y_train)

# The best model
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_:.2%}")

Fitting 4 folds for each of 72 candidates, totalling 288 fits
Best parameters: {'max_depth': 3, 'n_bits': 6, 'n_estimators': 5}
Best score: 86.07%
